In [10]:
# -*- coding: utf-8 -*-
import numpy as np
import lda
import csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models

from stemming.porter2 import stem
from nltk.stem import *
import unicodecsv
import re
import os
# import pyLDAvis.gensim
import gensim
import argparse

# Playing around with just dictionary words
# Using PyEnchant spell checker purpose
import enchant

In [11]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "paultan.csv"
args.stemmer = "lemma"
args.dictionary = "automotive"
args.num_iter = 20
args.num_top_words = 8
args.num_topics = 10
args.model = "dtm"
dir = os.getcwd()
model_dir = os.path.join(dir, 'models/')
dataset_dir = os.path.join(dir, 'datasets/')
dictionary_dir = os.path.join(dir, 'dictionaries/')
executable_dir = os.path.join(dir, 'executables/')
my_timeslices = [500,500,500,500,500, 346]

In [12]:
_digits = re.compile('\d')
def contains_digits(d):
    return bool(_digits.search(d))


def get_dict_dir(s):
  return os.path.join(dictionary_dir, s)

def get_exec_dir(s):
  return os.path.join(executable_dir, s)
d = enchant.Dict("en_US")
# Or using the /usr/share/dict/british-english word list
if args.dictionary != "none":
  with open(get_dict_dir(args.dictionary + "-english")) as word_file:
    english_words = set(word.strip().lower() for word in word_file)
    # print(english_words)
    def is_english_word(word):
      return word.lower() in english_words

def process_tokens(tokens,stemmer):
  tokens = [i for i in tokens if not i in en_stop and not contains_digits(i) and is_english_word(i)]
  if stemmer == 'porter':
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(i) for i in tokens]
  elif stemmer == 'porter2':
    tokens = [stem(i) for i in tokens]
  elif stemmer == 'lemma':
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(i) for i in tokens]
  return tokens


def get_model_with_arguments_filename():
  return (args.filename.split('.')[0] + "_" + args.stemmer + "_" + str(args.num_iter) +
   "_" + str(args.num_top_words) + "_" + str(args.num_topics)  + "_" + args.model)



if args.filename == "sample":
  X = lda.datasets.load_reuters()
  dictionary = lda.datasets.load_reuters_vocab()
  titles = lda.datasets.load_reuters_titles()
else:
  # X = np.zeros((len(contents), len(dictionary)), dtype=np.int)
  # for idx,i in enumerate(corpus):
  #   for j in i:
  #     X[idx][j[0]] = j[1]

  model_filename = os.path.join(model_dir, get_model_with_arguments_filename())
  print model_filename
  try:
    ldamodel = models.LdaModel.load(model_filename)
  except IOError:
    dataset_filepath = os.path.join(dataset_dir, args.filename)
    f = open(dataset_filepath)
    reader = unicodecsv.reader(f, encoding='utf-8')
    # csv_length = sum(1 for row in reader)
    # f.seek(0) #reset reader position
    identifiers = reader.next()
    contents_idx = identifiers.index("contents")
    title_idx = identifiers.index("title")

    contents = [ row[contents_idx] for row in reader if row[contents_idx] ]

    f.seek(0)
    reader.next()
    titles = [ row[title_idx] for row in reader if row[contents_idx] ]
    texts = list()
    tokenizer = RegexpTokenizer(r'\w+')
    en_stop = get_stop_words('en')
    for idx,i in enumerate(contents):
      if not idx % 10:
        print "INFO: Tokenizing articles <{}> ".format(idx)
      raw = i.lower()
      tokens = tokenizer.tokenize(raw)
      texts.append(process_tokens(tokens, args.stemmer))
      # print idx
      # add tokens to list

    print "[DEBUG] Length of Texts : {}".format(len(texts))
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]


/Users/Hii/Projects/news_scraper/models/paultan_lemma_20_8_10_dtm


In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
try:
  ldamodel = models.LdaModel.load(model_filename)
except IOError:
  if(args.model == "lda"):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=int(args.num_topics), id2word = dictionary, passes=int(args.num_iter))
  elif(args.model == "dtm"):
    ldamodel = gensim.models.wrappers.DtmModel(get_exec_dir('dtm-darwin64'), corpus, my_timeslices, num_topics=int(args.num_topics), id2word=dictionary,initialize_lda=True)
  else:
    raise ValueError('Unknown Model Type')
  ldamodel.save(model_filename)


# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
# pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)



In [15]:
  for idx, topic in enumerate(ldamodel.show_topics(topics=args.num_topics, topn=int(args.num_top_words),times=1, formatted=False)):
    print "Topic #" + str(idx) + " :",
    for word in topic:
      print word[1],
    print

Topic #0 : honda toyota nissan new type car year jaguar
Topic #1 : speed engine new feature wheel torque automatic control
Topic #2 : car vehicle driver kia hyundai feature safety new
Topic #3 : class highway new construction make bentley year turn
Topic #4 : bmw coupe new drive convertible car auto model
Topic #5 : ford mazda safety mitsubishi car fiat year mustang
Topic #6 : new engine fuel car sedan model design cylinder
Topic #7 : year new service increase suzuki price industry vehicle
Topic #8 : volkswagen audi porsche recall new scandal year model
Topic #9 : proton car racing race year ferrari formula design


In [13]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)



/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      33.197666        1       1 -0.153294 -0.013512
2      11.679065        1       2 -0.113325  0.078918
5       9.393395        1       3  0.107704 -0.102276
3       9.203218        1       4 -0.123469 -0.035756
9       8.071199        1       5  0.107808  0.122597
4       7.863333        1       6  0.080088 -0.025670
8       6.047470        1       7 -0.049261 -0.270745
0       5.407457        1       8  0.028666  0.169958
1       5.252834        1       9  0.271048 -0.009292
7       3.884363        1      10 -0.155967  0.085778, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
437   Default  1177.000000        toyota  1177.000000  30.0000  30.0000
355   Default  1343.000000           bmw  1343.000000  29.0000  29.0000
463   Default   741.000000    volkswagen   741.000000  28.0000  28.0000
436   Default   657.000000        nissan   657.000000  27.0000  27.0000
290   Default  1029.000000         honda  1029.000000  26.0000  26.0000
486   Default   695.000000         coupe   695.000000  25.0000  25.0000
338   Default   923.000000        proton   923.000000  24.0000  24.0000
282   Default   905.000000          fuel   905.000000  23.0000  23.0000
373   Default   857.000000          audi   857.000000  22.0000  22.0000
252   Default   993.000000         class   993.000000  21.0000  21.0000
27    Default  3609.000000           car  3609.000000  20.0000  20.0000
295   Default  4970.000000           new  4970.000000  19.0000  19.0000
206   Default   572.000000       service   572.000000  18.0000  18.0000
317   Default   657.000000         mazda   657.000000  17.0000  17.0000
351   Default  2316.000000        engine  2316.000000  16.0000  16.0000
168   Default   838.000000        safety   838.000000  15.0000  15.0000
500   Default   620.000000          ford   620.000000  14.0000  14.0000
37    Default  1196.000000        design  1196.000000  13.0000  13.0000
173   Default   442.000000       highway   442.000000  12.0000  12.0000
26    Default   406.000000          race   406.000000  11.0000  11.0000
395   Default   343.000000           kia   343.000000  10.0000  10.0000
78    Default  1362.000000       vehicle  1362.000000   9.0000   9.0000
156   Default  1931.000000          year  1931.000000   8.0000   8.0000
177   Default   410.000000       hyundai   410.000000   7.0000   7.0000
346   Default   342.000000       porsche   342.000000   6.0000   6.0000
389   Default   843.000000        driver   843.000000   5.0000   5.0000
46    Default   282.000000        racing   282.000000   4.0000   4.0000
493   Default   267.000000        suzuki   267.000000   3.0000   3.0000
251   Default   615.000000         sedan   615.000000   2.0000   2.0000
461   Default  1137.000000         power  1137.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
489   Topic10    40.288944        custom   118.275429   2.1713  -4.3415
163   Topic10     0.750953  interference     2.304476   2.1269  -8.3240
548   Topic10     7.178794         curve    22.978868   2.0848  -6.0664
119   Topic10    10.205365      ultimate    34.094593   2.0420  -5.7146
517   Topic10    16.771835       classic    60.126681   1.9715  -5.2178
12    Topic10     4.016310    protective    15.349369   1.9075  -6.6472
399   Topic10     0.750923       daytona     2.927248   1.8877  -8.3240
334   Topic10     1.397652  experimental     5.474048   1.8830  -7.7028
527   Topic10     3.675135           pit    16.618607   1.7393  -6.7360
130   Topic10    39.868561         style   217.160313   1.5532  -4.3520
37    Topic10   193.720038        design  1196.402429   1.4276  -2.7711
359   Topic10    19.698374         value   125.405269   1.3972  -5.0570
201   Topic10    27.824977       quality   191.281414   1.3204  -4.71